<a href="https://colab.research.google.com/github/jawdat23/Machine_Learning_Lab/blob/master/Jawdat_Haron_N26_ML_ENGINEERING_CHALLENGE_Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here is the code to the task 2 of the ML ENGINEERING CHALLENGE.
Hope you will enjoy it.


I am using Python 3.7.10 version and standard library. 

In [1]:
!python --version

Python 3.7.10



Here is the code snippet given by you to generate transactions and users csv files, no modifications, no additions.



In [2]:
# Copyright 2020 N26 GmbH

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.


import argparse
import csv
import datetime
import os
import random
import uuid


def generate_transactions(users):
    num_transactions = 100000
    num_users = len(users['data'])

    header = [
        'transaction_id',
        'date',
        'user_id',
        'is_blocked',
        'transaction_amount',
        'transaction_category_id'
    ]

    data = [[
        uuid.uuid4(),
        (datetime.date.today() - datetime.timedelta(days=random.randint(int(i / num_users), 100))).strftime('%Y-%m-%d'),
        users['data'][random.randint(0, num_users - 1)][0],
        random.random() < 0.99,
        '%.2f' % (random.random() * 100),
        random.randint(0, 10)
    ] for i in range(num_transactions)]

    return {'header': header, 'data': data}


def generate_users():
    num_users = 1000
    header = [
        'user_id',
        'is_active'
    ]

    data = [[
        uuid.uuid4(),
        random.random() < 0.9
    ] for _ in range(num_users)]

    return {'header': header, 'data': data}


def write_data(out, header, data):
    if os.path.exists(out):
        print('File %s already exists!' % out)
        return False

    try:
        with open(out, 'w') as f:
          writer = csv.writer(f)
          writer.writerow(header)
          writer.writerows(data)
    except Exception as err:
        print('Failed to write %s' % out)
        return False
    return True


if __name__ == '__main__':
    users = generate_users()
    transactions = generate_transactions(users)

    write_data('users.csv', users['header'], users['data'])
    write_data('transactions.csv', transactions['header'], transactions['data'])

Here is the code to generate the equivalent result of the SQL query.
since the task is just to write a python code to calculate the result, with no generalization, this means that the input are assumed to be in the exact same format and order as the files generated using your code.
That is why you will find raw numbers in the range [0-5].
To have a better understanding of these numbers and what they are referring to, and to see a well documented version of this code you may see the after next cell.

In [3]:
import csv
from itertools import groupby
from itertools import tee
from operator import itemgetter
from bisect import bisect_left
import sys

def print_results(results):
    sys.stdout.write('transaction_category_id \t sum_amount \t num_users\n')
    for result in results:
        sys.stdout.write(result[0] + '\t\t\t\t' + str('%-10.7f' % result[1])
         + '\t\t' + str(result[2]) + '\n')


def run(t_path, u_path):
  with open(t_path) as tf, open(u_path) as uf:
    transactions_reader = csv.reader(tf)
    users_reader = csv.reader(uf)
    active_users = [row[0] for row in users_reader if row[1] == "True"]
    active_users.sort(key=str)
    len_active_users = len(active_users)
    results = []
    for cat_id, cat_list in groupby(sorted([[row[5], row[4], row[2]] 
                                            for row in transactions_reader 
                                            if bisect_left(active_users, row[2]) != len_active_users
                                            and active_users[bisect_left(active_users, row[2])] == row[2]
                                            and row[3] == "False"]),
                                            key = lambda x : x[0]):
      cat_list, cat_list2 = tee(cat_list)
      results.append([cat_id, sum(float(v[1]) for v in cat_list),
                        len(set([v[2] for v in cat_list2]))])
    return sorted(results, key=itemgetter(1), reverse=True)


if __name__ == '__main__':   
  transactions_path = "transactions.csv"
  users_path = "users.csv"
  print_results(run(transactions_path, users_path))

transaction_category_id 	 sum_amount 	 num_users
8				4725.0800000		81
9				4506.4400000		78
10				4199.3500000		82
4				4186.7000000		78
7				3906.3400000		83
3				3684.1600000		75
2				3679.8800000		84
5				3674.5100000		77
0				3638.6200000		70
1				3605.4800000		73
6				3564.3700000		69


Here is the same code as the pervious one, but a bit generalized, where you can modify the `get_data` function to modify the indices of the csv files, and it will give you a hint of the data I am extracting and use to calculate this query.

In [4]:
#import useful tools from the python standard library
import csv
from itertools import groupby
from itertools import tee
from operator import itemgetter
from bisect import bisect_left
import sys

"""Gets the necessary data of the csv files

Args:
    None.

Returns:
    transactions_path: a string reffering to the path of the transactions csv file.
    users_path: a string reffering to the path of the users csv file.
    headers_data: a dictionary of two dictionaries about the indices of the two
                 csv files.

"""
def get_data():
  #define the paths of the csv files
  transactions_path = "transactions.csv"
  users_path = "users.csv"

  #define the indexes of the users and the transactions csv filea.
  headers_data = {'users' : {'user_id_index' : 0,
                             'is_active_index' : 1 },
                  'transactions' : {  'tran_id_index' : 0,
                                      'date_index' : 1,
                                      'user_id_index' : 2,
                                      'is_blocked_index' : 3,
                                      'tran_amount_index' : 4,
                                      'tran_cat_id_index' : 5}}
                                      
  return transactions_path, users_path, headers_data


"""Prints a list using sys stdout

Args:
    results: list of results, each result contains 3 elements 
    (transaction_category_id, sum_amount, and num_users)

Returns:
    None.
"""
def print_results(results):
    sys.stdout.write('transaction_category_id \t sum_amount \t num_users\n')
    for result in results:
        sys.stdout.write(result[0] + '\t\t\t\t' + str('%-10.7f' % result[1])
         + '\t\t' + str(result[2]) + '\n')


"""Gets a list of the transactions category ids with the sum of the transactions
   amount under this category, and the number of ditinct users that run these
   transactions sorted by the amount sum, where it takes only the non-blocked
   transactions and the active users into acount.

Args:
    t_path: a string reffering to the path of the transactions csv file.
    u_path: a string reffering to the path of the users csv file.
    h_data: a dictionary of two dictionaries about the indices of the two
                 csv files.
Returns:
    results: a list of the transactions category ids with the sum of the transactions
    amount under this category, and the number of ditinct users that run these
    transactions sorted by the amount sum.
"""
def run(t_path, u_path, h_data):
  #opening the files
  with open(t_path) as tf, open(u_path) as uf:

    #define the csv readers for the 2 files
    transactions_reader = csv.reader(tf)
    users_reader = csv.reader(uf)

    #get a copy of the active users
    active_users = [row[h_data['users']['user_id_index']] for row in users_reader
                    if row[h_data['users']['is_active_index']] == "True"]
    #sort the active users so it we can use binary search on it
    active_users.sort(key=str)
    #store the length of the active user
    len_active_users = len(active_users)
    #define the list that will carry the results
    results = []
    # This for loop is doing most of the work
    # First: extracting the transactions category id, amount, and user id for each
    #        row where the user id is in the active users id list and transaction is not blocked.
    # Second: sorting the list we obtained form the first step by the transaction category id.
    # Third: grouping the sorted list by the transaction category id, and get each group alone.
    for cat_id, cat_list in groupby(sorted([[row[h_data['transactions']['tran_cat_id_index']],
                                            row[h_data['transactions']['tran_amount_index']],
                                            row[h_data['transactions']['user_id_index']]] 
                                            for row in transactions_reader
                                            if bisect_left(active_users, row[h_data['transactions']['user_id_index']]) != len_active_users
                                            and active_users[bisect_left(active_users, row[h_data['transactions']['user_id_index']])] == row[h_data['transactions']['user_id_index']]
                                            and row[h_data['transactions']['is_blocked_index']] == "False"]),
                                            key = lambda x : x[0]):
      # create a copy of the generator that will yield us the data list of the category because we need to do 2 different operation on it.
      cat_list, cat_list2 = tee(cat_list)
      # calculating the sum of the transaction amount, and the number of distinct users in each category.
      results.append([cat_id, sum(float(v[1]) for v in cat_list),
                        len(set([v[2] for v in cat_list2]))])
    
    # return a sorted result list by the amonut sum
    return sorted(results, key=itemgetter(1), reverse=True)


if __name__ == '__main__':   
  transactions_path, users_path, headers_data = get_data()
  print_results(run(transactions_path, users_path, headers_data))

transaction_category_id 	 sum_amount 	 num_users
8				4725.0800000		81
9				4506.4400000		78
10				4199.3500000		82
4				4186.7000000		78
7				3906.3400000		83
3				3684.1600000		75
2				3679.8800000		84
5				3674.5100000		77
0				3638.6200000		70
1				3605.4800000		73
6				3564.3700000		69


some notes on the code: 

1.   I had to make a copy of the active users, as I have to iterate every time for each transaction and make sure its user is active, using a generator will only make me able to iterate once, and it is probably that the number of users is relevantly low, and it will not take much of a memory and storage.
2.   I tried to do the query using one run on the transaction reader iterator, so it will not take any huge memory or storage, and since it is one run the complexity will be lower also.
3.   The complexity of this code is O(M + Nlog(M)) where N is the number of the transactions, and M is the number of users, and in case M is low the complexity will be only O(N), taking in consideration that the number of categories in very low also.
4.   The method I am using make sure the headers will be filtered out, no need to manually remove them.
5.   In case we can make sure that the non-active users are not active for real. i.e. no empty rows or values other than 1 and 0, and we can make sure that the user_id in the transactions table is exisited for sure in the users table, then we can make a list of the non-active users only, which is 9 times smaller, and check if the user_id for the transaction is not in this list. This will make searching somehow faster, and the storage space needed is less.

